In [42]:
import numpy as np


def get_bernoulli_confidence_interval(values: np.array):
    """Вычисляет доверительный интервал для параметра распределения Бернулли.

    :param values: массив элементов из нулей и единиц.
    :return (left_bound, right_bound): границы доверительного интервала.
    """
    z = 1.96
    n = len(values)
    p = np.mean(values)
    K = np.sum(values)
    return (max(K / n - z * np.sqrt((K / n * (1 - K / n)) / n), 0), 
            min(K / n + z * np.sqrt((K / n * (1 - K / n)) / n), 1))

In [40]:
values = np.array([0,0,0,1,0,0,0,0])

In [43]:
get_bernoulli_confidence_interval(values)

(0, 0.3541765149399039)

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind


def estimate_first_type_error(df_pilot_group, df_control_group, metric_name, alpha=0.05, n_iter=10000, seed=None):
    """Оцениваем ошибку первого рода.

    Бутстрепим выборки из пилотной и контрольной групп тех же размеров, считаем долю случаев с значимыми отличиями.
    
    df_pilot_group - pd.DataFrame, датафрейм с данными пилотной группы
    df_control_group - pd.DataFrame, датафрейм с данными контрольной группы
    metric_name - str, названия столбца с метрикой
    alpha - float, уровень значимости для статтеста
    n_iter - int, кол-во итераций бутстрапа
    seed - int or None, состояние генератора случайных чисел.

    return - float, ошибка первого рода
    """
    pilot_indices = np.random.randint(0, len(df_pilot_group), (n_iter, len(df_pilot_group)))
    control_indices = np.random.randint(0, len(df_control_group), (n_iter, len(df_control_group)))
    
    pilot = df_pilot_group[metric_name].values[pilot_indices]
    control = df_control_group[metric_name].values[control_indices]
    
    res = 0
    #p_value = []
    for i in range(n_iter):
        res += int(ttest_ind(pilot[i], control[i]).pvalue < alpha)
        #p_value.append(ttest_ind(pilot[i], control[i]).pvalue)
        
    return res / n_iter#, p_value

In [20]:
df1 = pd.DataFrame({'metrica': [0.5, 0.45, 0.5],
'feature1': [1, 2, 3],
'feature2': [4, 5, 6]})


# Создаем второй датафрейм
df2 = pd.DataFrame({'metrica': [0.5, 0.5, 0.45],
'feature1': [7, 8, 9],
'feature2': [10, 11, 12]})


estimate_first_type_error(df1,df2, 'metrica')

0.0229

In [21]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind


def estimate_second_type_error(df_pilot_group, df_control_group, metric_name, effects, alpha=0.05, n_iter=10000, seed=None):
    """Оцениваем ошибки второго рода.

    Бутстрепим выборки из пилотной и контрольной групп тех же размеров, добавляем эффект к пилотной группе,
    считаем долю случаев без значимых отличий.
    
    df_pilot_group - pd.DataFrame, датафрейм с данными пилотной группы
    df_control_group - pd.DataFrame, датафрейм с данными контрольной группы
    metric_name - str, названия столбца с метрикой
    effects - List[float], список размеров эффектов ([1.03] - увеличение на 3%).
    alpha - float, уровень значимости для статтеста
    n_iter - int, кол-во итераций бутстрапа
    seed - int or None, состояние генератора случайных чисел

    return - dict, {размер_эффекта: ошибка_второго_рода}
    """
    np.random.seed(seed)
    
    pilot_indices = np.random.randint(0, len(df_pilot_group), (n_iter, len(df_pilot_group)))
    control_indices = np.random.randint(0, len(df_control_group), (n_iter, len(df_control_group)))
    
    #pilot = df_pilot_group[metric_name].values[pilot_indices]
    control = df_control_group[metric_name].values[control_indices]
    
    second_type_error = {}
    for effect in effects:
        res = 0
        pilot_with_effect = (df_pilot_group[metric_name] * np.random.normal(effect, 
                                                                            0.005, 
                                                                            len(df_pilot_group)
                                                                           )
                            ).values[pilot_indices]
        for i in range(n_iter):
            res += int(ttest_ind(pilot_with_effect[i], control[i]).pvalue < alpha)
            
        second_type_error[effect] = 1 - res / n_iter
        
    return second_type_error

In [22]:
estimate_second_type_error(df1,df2, 'metrica', [1.03, 1.1])

[[0.51718789 0.51442818 0.46673997]
 [0.46673997 0.51718789 0.46673997]
 [0.51718789 0.51442818 0.46673997]
 ...
 [0.46673997 0.46673997 0.46673997]
 [0.51718789 0.51718789 0.46673997]
 [0.46673997 0.51442818 0.46673997]]
[[0.55171024 0.55608927 0.49620483]
 [0.49620483 0.55171024 0.49620483]
 [0.55171024 0.55608927 0.49620483]
 ...
 [0.49620483 0.49620483 0.49620483]
 [0.55171024 0.55171024 0.49620483]
 [0.49620483 0.55608927 0.49620483]]


{1.03: 0.8065, 1.1: 0.6658999999999999}

In [29]:
import numpy as np
import pandas as pd


def select_stratified_groups(data, strat_columns, group_size, weights=None, seed=None):
    """Подбирает стратифицированные группы для эксперимента.

    data - pd.DataFrame, датафрейм с описанием объектов, содержит атрибуты для стратификации.
    strat_columns - List[str], список названий столбцов, по которым нужно стратифицировать.
    group_size - int, размеры групп.
    weights - dict, словарь весов страт {strat: weight}, где strat - либо tuple значений элементов страт,
        например, для strat_columns=['os', 'gender', 'birth_year'] будет ('ios', 'man', 1992), либо просто строка/число.
        Если None, определить веса пропорционально доле страт в датафрейме data.
    seed - int, исходное состояние генератора случайных чисел для воспроизводимости
        результатов. Если None, то состояние генератора не устанавливается.

    return (data_pilot, data_control) - два датафрейма того же формата, что и data
        c пилотной и контрольной группами.
    """
    # YOUR_CODE_HERE
    np.random.seed(seed=seed)
    if weights == None:
        weights_df = data.groupby(strat_columns, as_index=False).size()
        weights_df['weight'] = weights_df['size'] / weights_df['size'].sum()

        weights = {}
        for i in range(len(weights_df)):
            curr_row = weights_df.iloc[i]
            if len(strat_columns) > 1:
                weights[tuple(curr_row[col] for col in strat_columns)] = curr_row.weight
            else:
                weights[curr_row[strat_columns[0]]] = curr_row.weight
            
    data_pilot = data.head(0)
    data_control = data.head(0)

    for cols, val in weights.items():
        str_group_size = int(group_size * val + 0.5)
        
        data_curr = data   
        if type(cols) == tuple:
            for col_n, col_val in enumerate(cols):
                data_curr = data_curr[data_curr[strat_columns[col_n]]==col_val]
        else: 
            data_curr = data_curr[data_curr[strat_columns[0]]==cols]
        
        index = np.arange(0, len(data_curr))
        rand_index = np.random.choice(index, size=2*str_group_size, replace=False)
        pilot_rand_index = rand_index[:str_group_size]
        control_rand_index = rand_index[str_group_size:]

        data_pilot = pd.concat([data_pilot, data_curr.iloc[pilot_rand_index]])
        data_control = pd.concat([data_control, data_curr.iloc[control_rand_index]])
    return data_pilot, data_control
    
        
        
        

In [30]:
np.random.default_rng().multinomial(100, [0.1])

array([100])

In [31]:
sample_sizes = np.zeros(shape=len(strata))

NameError: name 'strata' is not defined

In [32]:
strata_sizes = [len(stratum) for stratum in strata]
full_size = np.sum(strata_sizes)
weights = np.array(strata_sizes) / full_size

sample_sizes = np.zeros(shape=len(strata))

NameError: name 'strata' is not defined

In [33]:
На вход подаются:

Датафрейм с описанием объектов, которые нужно стратифицировать;
Название параметров, по которым нужно стратифицировать:
Размер групп;
Словарь весов страт (если None, то веса нужно взять пропорционально доле страт в датафрейме);
Состояние генератора случайных чисел.
Функция должна возвращать пару датафреймов того же формата, что она получила на вход. 
Доли страт в каждом датафрейме должны в точности совпадать друг с другом и могут немного отличаться от переданных 
в функцию или посчитанных на основе входных данных, но не более чем на 2 / group_size 
(предполагается, что размеры страт достаточно велики, чтобы это всегда было реализуемо).

При повторном вызове функции при seed=None полученные группы должны отличаться от вернувшихся 
при прошлом вызове функции (необходимо использовать рандомизацию).

SyntaxError: invalid syntax (3621733661.py, line 1)

In [34]:
import pandas as pd
data = pd.DataFrame({#'metrica': [0.5, 0.45, 0.5,1,1,1,1,1,1],
'feature1': [2, 2, 2,2,0,2,2,2,2] * 100 + [0,0],
'feature2': [4, 5, 5,5,4,5,4,5,4] * 100 + [5,4]})

In [35]:
data1, data2 = select_stratified_groups(data, strat_columns=['feature1'], group_size=100, weights=None, seed=None)

In [36]:
data1.groupby(strat_columns, as_index=False).size()

,feature1,feature2,size
0,0,4,11
1,2,4,33
2,2,5,56


In [12]:
data1.groupby(strat_columns, as_index=False).size()

,feature1,feature2,size
0,0,4,11
1,2,4,33
2,2,5,55


In [9]:
strat_columns = ['feature1', 'feature2']
weights = None

In [13]:
weights_df = data.groupby(strat_columns, as_index=False).size()

weights_df['weight'] = weights_df['size'] / weights_df['size'].sum()

#group_size = 100
#weights['str_group_size'] = (group_size * weights['weight']).astype(int)

#weights = weights.sort_values('str_group_size')

In [14]:
weights_df

,feature1,feature2,size,weight
0,0,4,101,0.111973
1,0,5,1,0.001109
2,2,4,300,0.332594
3,2,5,500,0.554324


In [114]:
weights = {}
for i in range(len(weights_df)):
    curr_row = weights_df.iloc[i]
    weights[tuple(curr_row[col] for col in strat_columns)] = curr_row.weight

In [116]:
weights.items()

dict_items([((0.0, 5.0), 0.0011098779134295228), ((2.0, 4.0), 0.4439511653718091), ((2.0, 5.0), 0.5549389567147613)])

In [23]:
curr_row.weight

0.3333333333333333

In [28]:
tuple(curr_row[col] for col in strat_columns)

(2.0, 4.0)

In [112]:
if weights == None:
    weights = data.groupby(strat_columns, as_index=False).size()

    weights['weight'] = weights['size'] / weights['size'].sum()

    group_size = 100
    weights['str_group_size'] = (group_size * weights['weight']).astype(int)

    weights = weights.sort_values('str_group_size')

    data_pilot = data.head(0)
    data_control = data.head(0)

    for i in range(len(weights)):
        str_size = weights['size'].iloc[i]
        str_group_size = weights['str_group_size'].iloc[i]
        index = np.arange(0, str_size)
        rand_index = np.random.choice(index, size=2*str_group_size, replace=False)
        pilot_rand_index = rand_index[:str_group_size]
        control_rand_index = rand_index[str_group_size:]

        str_data = data.merge(weights[strat_columns].iloc[[i]], on=strat_columns)
        data_pilot = pd.concat([data_pilot, str_data.iloc[pilot_rand_index]])
        data_control = pd.concat([data_control, str_data.iloc[control_rand_index]])
    

In [115]:
def calc_strat_stats(
    strata: list,
    sample_size=100, n_iter=1000,
    is_stratified=True
):
    """Считаем дисперсию среднего значения при стратифицированном сэмплировании.
    
    strata - список страт
    sample_size - размер сэмплируемой выборки
    n_iter - кол-во итераций сэмплирования
    is_stritified - флаг стратификационного семплирования (сохраняем ли мы пропорции страт)
    
    return: среднее средних, дисперсия средних
    """
    
    strata_sizes = [len(stratum) for stratum in strata]
    full_size = np.sum(strata_sizes)
    weights = np.array(strata_sizes) / full_size
    
    sample_sizes = np.zeros(shape=len(strata))
    if is_stratified:
        sample_sizes = (weights * sample_size + 0.5).astype(int)
    else:
        while (np.array(sample_sizes)).min() == 0:
            sample_sizes = np.random.default_rng().multinomial(sample_size, weights)

    assert sample_sizes.min() != 0
    
    means = []
    for _ in range(n_iter):
        strata_means = [get_sample_mean(stratum, size) for stratum, size in zip(strata, sample_sizes)]
        means.append((weights * np.array(strata_means)).sum())
    return np.mean(means), np.var(means)

,feature1,feature2,size,weight,str_group_size
2,3,4,100,0.111111,11
3,3,5,200,0.222222,22
0,2,4,300,0.333333,33
1,2,5,300,0.333333,33


In [113]:
data_control

,feature1,feature2
60,3,4
63,3,4
22,3,4
41,3,4
52,3,4
...,...,...
17,2,5
175,2,5
14,2,5
248,2,5


In [114]:
data_pilot

,feature1,feature2
49,3,4
38,3,4
0,3,4
57,3,4
20,3,4
...,...,...
276,2,5
297,2,5
289,2,5
184,2,5


In [75]:
import numpy as np
np.random.randint(low=0, high=30, size=2*10)

array([ 0, 10,  5, 16, 13, 21,  8, 22, 17,  8, 26, 20, 24,  3,  1,  8, 11,
       20, 21,  4])

In [ ]:
numpy.random.choice(np

In [80]:
rand_index[:str_group_size]

array([ 70, 264, 248, 101, 294, 289,  62, 192, 292, 166,  19, 174,  93,
       251,  52, 250, 267,   4, 143, 212, 211, 152,  91, 109,  68, 151,
        41, 221, 235, 223, 219, 114,  79])

In [81]:
rand_index[str_group_size:]

array([177, 135, 273, 183, 195,  83,   9,  90, 215,   6, 254, 120,  95,
        39,  87, 231, 270,  47, 266, 100, 225, 112,  88, 234, 190, 255,
       291,  63, 272, 198,  64,  65, 181])

In [82]:
weights

,feature1,feature2,size,weight,str_group_size
2,3,4,100,0.111111,11
3,3,5,200,0.222222,22
0,2,4,300,0.333333,33
1,2,5,300,0.333333,33



Задача. Вычисление метрик для CUPED.
Наша цель — научиться получать преобразованную с помощью CUPED метрику.

Для этого нужно будет написать две функции.

Первая функция вспомогательная, она будет вычислять значения целевой метрики для пользователей на произвольном промежутке времени. Первую функцию можно будет использовать, чтобы посчитать значения метрики пользователей во время пилота и до пилота.

Вторая функция будет вычислять непосредственно преобразованную cuped-метрику. В качестве ковариаты будем использовать значение метрики, посчитанное на периоде до начала пилота.

Целевая метрика — суммарная стоимость покупок пользователя за определённый период времени.



In [133]:
import numpy as np
import pandas as pd


def calculate_metric(
    df, value_name, user_id_name, list_user_id, date_name, period, metric_name
):
    """Вычисляет значение метрики для списка пользователей в определённый период.
    
    df - pd.DataFrame, датафрейм с данными
    value_name - str, название столбца со значениями для вычисления целевой метрики
    user_id_name - str, название столбца с идентификаторами пользователей
    list_user_id - List[int], список идентификаторов пользователей, для которых нужно посчитать метрики
    date_name - str, название столбца с датами
    period - dict, словарь с датами начала и конца периода, за который нужно посчитать метрики.
        Пример, {'begin': '2020-01-01', 'end': '2020-01-08'}. Дата начала периода входит нужный
        полуинтервал, а дата окончание нет, то есть '2020-01-01' <= date < '2020-01-08'.
    metric_name - str, название полученной метрики

    return - pd.DataFrame, со столбцами [user_id_name, metric_name], кол-во строк должно быть равно
        кол-ву элементов в списке list_user_id.
    """
    # YOUR_CODE_HERE
    df_subsample = df[(df[user_id_name].isin(list_user_id))&
                      (df[date_name]>=period['begin'])&
                      (df[date_name]<period['end'])]
    result = df_subsample.groupby(user_id_name, as_index=False)[value_name].agg(func='sum')
    result = pd.DataFrame(list_user_id, columns=[user_id_name]).merge(result, on=user_id_name, how='left').fillna(0)
    result.columns = [user_id_name, metric_name]
    return result



def calculate_metric_cuped(
    df, value_name, user_id_name, list_user_id, date_name, periods, metric_name
):
    """Вычисляет метрики во время пилота, коварианту и преобразованную метрику cuped.
    
    df - pd.DataFrame, датафрейм с данными
    value_name - str, название столбца со значениями для вычисления целевой метрики
    user_id_name - str, название столбца с идентификаторами пользователей
    list_user_id - List[int], список идентификаторов пользователей, для которых нужно посчитать метрики
    date_name - str, название столбца с датами
    periods - dict, словарь с датами начала и конца периода пилота и препилота.
        Пример, {
            'prepilot': {'begin': '2020-01-01', 'end': '2020-01-08'},
            'pilot': {'begin': '2020-01-08', 'end': '2020-01-15'}
        }.
        Дата начала периода входит в полуинтервал, а дата окончания нет,
        то есть '2020-01-01' <= date < '2020-01-08'.
    metric_name - str, название полученной метрики

    return - pd.DataFrame, со столбцами
        [user_id_name, metric_name, f'{metric_name}_prepilot', f'{metric_name}_cuped'],
        кол-во строк должно быть равно кол-ву элементов в списке list_user_id.
    """
    # YOUR_CODE_HERE
    
    
    prepilot = calculate_metric(df, value_name, user_id_name, list_user_id, date_name, periods['prepilot'], metric_name)
    pilot = calculate_metric(df, value_name, user_id_name, list_user_id, date_name, periods['pilot'], metric_name)
    prepilot.columns = [user_id_name, f'{metric_name}_prepilot']
    
    result = pilot.merge(prepilot, on=user_id_name, how='inner')
    
    covariance = np.cov(result[f'{metric_name}_prepilot'], result[metric_name])[0, 1]
    variance = np.var(result[f'{metric_name}_prepilot'])
    theta = covariance / variance
    
    result[f'{metric_name}_cuped'] = result[metric_name] - theta * result[f'{metric_name}_prepilot']
    
    return result

In [134]:
pd.date_range('2020-01-01', periods=1000, freq='D')

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08',
               '2020-01-09', '2020-01-10',
               ...
               '2022-09-17', '2022-09-18', '2022-09-19', '2022-09-20',
               '2022-09-21', '2022-09-22', '2022-09-23', '2022-09-24',
               '2022-09-25', '2022-09-26'],
              dtype='datetime64[ns]', length=1000, freq='D')

In [135]:
pd.concat([pd.Series(pd.date_range('2020-01-01', periods=1000, freq='D')), 
           pd.Series(pd.date_range('2020-01-01', periods=1000, freq='D')), 
           pd.Series(pd.date_range('2020-01-01', periods=1000, freq='D')), 
           pd.Series(pd.date_range('2020-01-01', periods=1000, freq='D'))
          ])

0     2020-01-01
1     2020-01-02
2     2020-01-03
3     2020-01-04
4     2020-01-05
         ...    
995   2022-09-22
996   2022-09-23
997   2022-09-24
998   2022-09-25
999   2022-09-26
Length: 4000, dtype: datetime64[ns]

In [136]:
a = np.random.randint(1, 1000, 2000)
b = np.random.randint(1, 1000, 2000)


df = pd.DataFrame({
    'value_name': np.hstack((a + 5, a)),
    'user_id_name': np.hstack((b, b)),
    'date_name': pd.concat([pd.Series(pd.date_range('2020-01-01', periods=1000, freq='D')), 
           pd.Series(pd.date_range('2020-01-01', periods=1000, freq='D')), 
           pd.Series(pd.date_range('2020-01-01', periods=1000, freq='D')), 
           pd.Series(pd.date_range('2020-01-01', periods=1000, freq='D'))
          ]),
})

list_user_id = np.random.choice(np.arange(1, 1200), size=500, replace=False)
period = {'begin': '2020-01-01', 'end': '2022-06-03'}


calculate_metric(df=df, 
                 value_name='value_name', 
                 user_id_name='user_id_name', 
                 list_user_id=list_user_id, 
                 date_name='date_name', 
                 period=period, 
                 metric_name='sum_values')

,user_id_name,sum_values
0,36,1627.0
1,616,0.0
2,1169,0.0
3,558,0.0
4,1100,0.0
...,...,...
495,344,1377.0
496,780,0.0
497,360,3633.0
498,910,11.0


In [137]:
df

,value_name,user_id_name,date_name
0,270,661,2020-01-01
1,862,367,2020-01-02
2,568,202,2020-01-03
3,794,696,2020-01-04
4,464,794,2020-01-05
...,...,...,...
995,986,390,2022-09-22
996,915,797,2022-09-23
997,400,930,2022-09-24
998,734,357,2022-09-25


In [138]:
period = {
            'prepilot': {'begin': '2020-01-01', 'end': '2020-07-01'},
            'pilot': {'begin': '2020-07-01', 'end': '2021-01-01'}
            #'pilot': {'begin': '2020-01-01', 'end': '2020-08-08'}
        }


calculate_metric_cuped(df=df, 
                 value_name='value_name', 
                 user_id_name='user_id_name', 
                 list_user_id=list_user_id, 
                 date_name='date_name', 
                 periods=period, 
                 metric_name='sum_values').reset_index(drop=True) 

,user_id_name,sum_values,sum_values_prepilot,sum_values_cuped
0,36,57.0,1307.0,-6.023969
1,616,0.0,0.0,0.000000
2,1169,0.0,0.0,0.000000
3,558,0.0,0.0,0.000000
4,1100,0.0,0.0,0.000000
...,...,...,...,...
495,344,0.0,1377.0,-66.399392
496,780,0.0,0.0,0.000000
497,360,0.0,1439.0,-69.389052
498,910,11.0,0.0,11.000000


In [113]:
prepilot = calculate_metric(df=df, 
                 value_name='value_name', 
                 user_id_name='user_id_name', 
                 list_user_id=list_user_id, 
                 date_name='date_name', 
                 period=period['prepilot'], 
                 metric_name='sum_values')
pilot = calculate_metric(df=df, 
                 value_name='value_name', 
                 user_id_name='user_id_name', 
                 list_user_id=list_user_id, 
                 date_name='date_name', 
                 period=period['pilot'], 
                 metric_name='sum_values')

metric_name='sum_values'
user_id_name='user_id_name'
covariance = np.cov(prepilot[metric_name], pilot[metric_name])[0, 1]
variance = prepilot[metric_name].var()
theta = covariance / variance

result = pd.DataFrame(list_user_id, columns=[user_id_name]).merge(pilot, on=user_id_name, how='left').merge(prepilot, on=user_id_name, how='left')


In [79]:
result

,user_id_name,sum_values_x,sum_values_y
0,10,222.0,0.0
1,309,0.0,0.0
2,64,134.0,0.0
3,530,0.0,0.0
4,156,0.0,0.0
5,217,0.0,0.0
6,225,0.0,0.0
7,85,250.0,0.0
8,769,0.0,0.0
9,274,454.0,0.0


In [75]:
prepilot[metric_name]

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
9    0.0
Name: sum_values, dtype: float64

In [76]:
pilot[]

SyntaxError: invalid syntax (4113088055.py, line 1)

In [29]:
theta

1.0000000000000004

In [34]:
pd.DataFrame(list_user_id, columns=['f'])

,f
0,124
1,170
2,573
3,941
4,538
5,156
6,679
7,736
8,217
9,662
